In [1]:
import torch
from itertools import combinations
import numpy as np

In [30]:
m = torch.tensor([[-1, 0, 0]], dtype=torch.float32)
rp = torch.tensor([[0, 0., 1.]])
v = torch.tensor([[1, 1.5, 1.5, 1.5]])
p = torch.tensor([[[0, 0, 0],
                   [2, 2, 2],
                   [-2, 2, 2],
                   [0, -2, 2]]], dtype=torch.float32)

In [31]:
p.shape

torch.Size([1, 4, 3])

In [32]:
bs = m.shape[0]

1. Formar K

In [33]:
k = torch.cat((torch.ones((1,1,4)), torch.transpose(p, 1, 2)), dim=1).inverse()

2. Calcular plano

In [34]:
ab = torch.bmm(torch.transpose(k, 1, 2), v.unsqueeze(-1))
a = ab[:,0,0]
b = ab[:, 1:, 0]

In [35]:
a,b

(tensor([1.]), tensor([[0.0000, 0.0000, 0.2500]]))

3. Evaluar n

In [36]:
n = b/b.norm()

4. Calcular q, R, rc

In [37]:
def batch_dot(v1, v2):
    return torch.einsum('b d, b d -> b', v1, v2)

In [38]:
mn = torch.cross(m, n)
q = mn / torch.sqrt(batch_dot(mn, mn))

In [39]:
nq = torch.cross(n, q)
mn_dot = batch_dot(m, n)
rc = rp - (batch_dot(rp, n) + a/b.norm())*(n - (mn_dot * nq) / batch_dot(m, nq))

In [40]:
R = rc - rp

5. Calcular M_L, Q_L

In [41]:
i = p[:, 1]
j = p[:, 2]
k = p[:, 3]
M_L0 = (j[:, 1] - i[:, 1]) * (k[:, 2] - i[:, 2]) - (k[:, 1] - i[:, 1]) * (j[:, 2] - i[:, 2])
M_L1 = (j[:, 2] - i[:, 2]) * (k[:, 0] - i[:, 0]) - (k[:, 2] - i[:, 2]) * (j[:, 0] - i[:, 0])
M_L2 = (j[:, 0] - i[:, 0]) * (k[:, 1] - i[:, 1]) - (k[:, 0] - i[:, 0]) * (j[:, 1] - i[:, 1])
M_L = torch.stack((M_L0, M_L1, M_L2), dim=1)
Q_L = - batch_dot(i, M_L)

6. Resolver la ecuación para sacar el ángulo

In [42]:
c1 = - batch_dot(M_L, R)
c2 = R.norm() * batch_dot(M_L, m)
c3 = batch_dot(M_L, rc) + Q_L

In [43]:
c1, c2, c3

(tensor([80.]), tensor([0.]), tensor([-96.]))

In [44]:
if c1 == c3:
    phi1 = -2*torch.atan(c1/c2)
    phi2 = phi1
else:
    phi1 = 2 * torch.atan((c2 + torch.sqrt(c1.pow(2) + c2.pow(2) - c3.pow(2)))/(c1-c2))
    phi2 = 2 * torch.atan((c2 - torch.sqrt(c1.pow(2) + c2.pow(2) - c3.pow(2)))/(c1-c2))

Ahora en formato bucle:

In [45]:
wrap_to_2pi = lambda x: x % (2 * np.pi) + (2 * np.pi) * (x == 0)

In [46]:
phiR = torch.zeros((bs, 4))
combs = list(combinations(range(4), 3))
for it, (ii, ij, ik) in enumerate(combs):
    print(ii, ij, ik)
    i = p[:, ii]
    j = p[:, ij]
    k = p[:, ik]
    
    M_L0 = (j[:, 1] - i[:, 1]) * (k[:, 2] - i[:, 2]) - (k[:, 1] - i[:, 1]) * (j[:, 2] - i[:, 2])
    M_L1 = (j[:, 2] - i[:, 2]) * (k[:, 0] - i[:, 0]) - (k[:, 2] - i[:, 2]) * (j[:, 0] - i[:, 0])
    M_L2 = (j[:, 0] - i[:, 0]) * (k[:, 1] - i[:, 1]) - (k[:, 0] - i[:, 0]) * (j[:, 1] - i[:, 1])
    M_L = torch.stack((M_L0, M_L1, M_L2), dim=1)
    Q_L = - batch_dot(i, M_L)
    
    c1 = - batch_dot(M_L, R)
    c2 = R.norm() * batch_dot(M_L, m)
    c3 = batch_dot(M_L, rc) + Q_L
    
    if c1 == c3:
        phi1 = -2*torch.atan(c1/c2)
        phi2 = phi1
    else:
        phi1 = 2 * torch.atan((c2 + torch.sqrt(c1.pow(2) + c2.pow(2) - c3.pow(2)))/(c1-c3))
        phi2 = 2 * torch.atan((c2 - torch.sqrt(c1.pow(2) + c2.pow(2) - c3.pow(2)))/(c1-c3))
    #print(phi1,phi2)
    phi1 = wrap_to_2pi(phi1)
    phi2 = wrap_to_2pi(phi2)
    print(phi1,phi2)
    phiR[:, it] = min(phi1, phi2)

0 1 2
tensor([0.6435]) tensor([5.6397])
0 1 3
tensor([6.1854]) tensor([2.3121])
0 2 3
tensor([0.0978]) tensor([3.9711])
1 2 3
tensor([nan]) tensor([nan])


In [47]:
phiR = torch.nan_to_num(phiR, nan=10)
phiE, idx_face = torch.min(phiR, dim=1)

In [48]:
combs[idx_face]

(0, 2, 3)

In [49]:
phiE

tensor([0.0978])

7. Punto de salida y dirección

In [50]:
re = rc - torch.cos(phiE) * R + R.norm()*torch.sin(phiE)*m

In [51]:
re

tensor([[-0.4881,  0.0000,  0.9761]])

In [52]:
me = torch.cos(phiE)*m + torch.sin(phiE)/R.norm() * R

In [53]:
me

tensor([[-0.9952,  0.0000, -0.0976]])

In [54]:
def simulate_tetrahedron(tetra, m, rp):
    k = tetra['k']
    a, b = tetra['speed_field']
    p = tetra['p']
    n = tetra['n']
    
    mn = torch.cross(m, n)
    q = mn / torch.sqrt(batch_dot(mn, mn))
    nq = torch.cross(n, q)
    mn_dot = batch_dot(m, n)
    rc = rp - (batch_dot(rp, n) + a/b.norm())*(n - (mn_dot * nq) / batch_dot(m, nq))
    R = rc - rp
    
    phiR = torch.zeros((bs, 4))
    combs = list(combinations(range(4), 3))
    for it, (ii, ij, ik) in enumerate(combs):
        i = p[:, ii]
        j = p[:, ij]
        k = p[:, ik]

        M_L0 = (j[:, 1] - i[:, 1]) * (k[:, 2] - i[:, 2]) - (k[:, 1] - i[:, 1]) * (j[:, 2] - i[:, 2])
        M_L1 = (j[:, 2] - i[:, 2]) * (k[:, 0] - i[:, 0]) - (k[:, 2] - i[:, 2]) * (j[:, 0] - i[:, 0])
        M_L2 = (j[:, 0] - i[:, 0]) * (k[:, 1] - i[:, 1]) - (k[:, 0] - i[:, 0]) * (j[:, 1] - i[:, 1])
        M_L = torch.stack((M_L0, M_L1, M_L2), dim=1)
        Q_L = - batch_dot(i, M_L)

        c1 = - batch_dot(M_L, R)
        c2 = R.norm() * batch_dot(M_L, m)
        c3 = batch_dot(M_L, rc) + Q_L

        if c1 == c3:
            phi1 = -2*torch.atan(c1/c2)
            phi2 = phi1
        else:
            phi1 = 2 * torch.atan((c2 + torch.sqrt(c1.pow(2) + c2.pow(2) - c3.pow(2)))/(c1-c3))
            phi2 = 2 * torch.atan((c2 - torch.sqrt(c1.pow(2) + c2.pow(2) - c3.pow(2)))/(c1-c3))
        phi1 = wrap_to_2pi(phi1)
        phi2 = wrap_to_2pi(phi2)
        phiR[:, it] = min(phi1, phi2)
    
    phiR = torch.nan_to_num(phiR, nan=10)
    phiE, idx_face = torch.min(phiR, dim=1)
    re = rc - torch.cos(phiE) * R + R.norm()*torch.sin(phiE)*m
    me = torch.cos(phiE)*m + torch.sin(phiE)/R.norm() * R
    
    return re, me, idx_face